## 1) Downloading Time Series Data with Polygon-API-Client

To start training a model with stock prices and volume to predict the next minute price, we first need to collect and prepare our dataset. In this case, we're interested in downloading 3 months of intraday data with 1-minute intervals using the Polygon API. Let's dive into how to achieve this step by step.



### Prerequisites

Before you begin, ensure you have created a Polygon.io account and installed the necessary Python packages:

- `numpy`: For efficient manipulation of large arrays.
- `pandas`: For handling and manipulating the dataset.
- `polygon-api-client`: To fetch stock market data from Polygon.io.

1) Create a **FREE** Polygon.io account at the their sign up page: [https://polygon.io/dashboard/signup](https://polygon.io/dashboard/signup)

2) Get your Polygon.io API key at [https://polygon.io/dashboard/api-keys](https://polygon.io/dashboard/api-keys)

3) You can install these packages using pip:


In [ ]:
!pip install numpy pandas polygon-api-client


### Fetching the Data

To fetch the data, you'll need an API key from Polygon.io. Once you have your key, you can use the following code snippet to download the stock price data:


In [ ]:

from datetime import datetime, timedelta
from polygon import RESTClient
import pandas as pd


def fetch_data(symbol:str, start_date:datetime, end_date:datetime, polygon_client: RESTClient):
    aggregate_iter = polygon_client.list_aggs(
        symbol, 1, "minute", start_date, end_date, limit=50_000)
    
    # Initialize an empty list to store the data
    data = []
    
    # Loop through the returned data, extracting the necessary information
    for aggregate in aggregate_iter:
        data.append([
            # Convert timestamp from milliseconds
            pd.to_datetime(aggregate.timestamp, unit='ms'),
            aggregate.open,  # Open price
            aggregate.high,  # High price
            aggregate.low,  # Low price
            aggregate.close,  # Close price
            aggregate.volume,  # Volume
            aggregate.transactions,  # Number of transactions in the aggregate window
            aggregate.vwap,  # Volume weighted average price
        ])
    
    # Create a DataFrame
    columns = ['timestamp', 'open', 'high', 'low',
               'close', 'volume', "transactions", "vwap"]
    df = pd.DataFrame(data, columns=columns)
    return df


# Example usage
api_key = "YOUR_API_KEY_HERE"
symbol = "AAPL"
end_date = datetime.now()
start_date = end_date - timedelta(days=120)


client = RESTClient(api_key)
df = fetch_data(symbol, start_date, end_date, client)
print(df.head())


Here's an updated explanation of the code you provided, which fetches stock price data using the Polygon API:

- **Datetime and timedelta usage**: This code imports `datetime` and `timedelta` from the datetime module to calculate the start and end dates for the data fetching. This allows for dynamic date calculations, such as fetching the last 90 days of data up to the current date.
  
- **Polygon RESTClient**: The `RESTClient` from the polygon package is initialized outside of the `fetch_data` function and passed as an argument. This allows for more flexible usage of the client, such as reusing the same client for multiple data fetch operations without reinitializing it each time.
  
- **Fetching data with list_aggs method**: Instead of `stocks_equities_aggregates`, this code uses the `list_aggs` method to fetch aggregated stock data. This method is called with the stock symbol, aggregation size (1 minute), and the start and end dates. The `limit=50_000` parameter specifies the maximum number of results to return, accommodating large datasets.
  
- **Data extraction and conversion**: As before, the code loops through each aggregate in the fetched data. However, it now also extracts the `transactions` (the number of transactions within each minute) and `vwap` (volume-weighted average price), providing a more detailed dataset.
  
- **DataFrame creation**: The extracted data is stored in a list and then used to create a pandas DataFrame. This DataFrame includes columns for timestamp, open, high, low, close, volume, transactions, and vwap. This structure facilitates easy manipulation and analysis of the stock data.


### Try it Out

Experiment with fetching data for different symbols or adjusting the date range to explore how the market has performed over different periods. For instance, you can change the `symbol` to "AAPL" for Apple Inc. or adjust the `timedelta` to fetch data over a different duration. 


In [ ]:
# Adjust the symbol or date range
start_date = end_date - timedelta(days=180)  # Example: Change to fetch the last 180 days
df = fetch_data(symbol, start_date, end_date, client)
print(df.head())


In the next steps, we will preprocess this data to be suitable for training our model with TensorFlow and Scikit-Learn, focusing on using stock prices and volume to predict the next minute price.

## 2) Plotting Candlestick Charts with Matplotlib

Candlestick charts are a popular method for visualizing stock price movements over time. Each candlestick provides information on the open, high, low, and close prices for a given time period. In this section, we'll show how to plot candlestick charts using Matplotlib and the data we've fetched in the previous step.



### Prerequisites

First, ensure you have the necessary package for plotting:

- `matplotlib`: For creating visualizations in Python.
- `mplfinance`: A Matplotlib utility specifically for financial data visualization, including candlestick charts.

You can install these packages using pip:


In [ ]:
!pip install matplotlib mplfinance

### Preparing the Data

Before plotting, ensure your DataFrame is correctly formatted for `mplfinance`. The DataFrame's index must be a DatetimeIndex, so we'll set the 'timestamp' column as the index:


In [ ]:
df.set_index('timestamp', inplace=True)


### Plotting the Candlestick Chart

Now, let's plot the candlestick chart for our stock data:


In [ ]:
import mplfinance as mpf

last_timestamp = df.index.max()  # Get the most recent timestamp
# Calculate one hour before the last timestamp
one_hour_ago = last_timestamp - pd.Timedelta(hours=1)

# Filter the DataFrame for the last hour
df_last_hour = df[one_hour_ago:]

# Plotting
mpf.plot(df_last_hour, type='candle', style='charles',
         title='Stock Price Candlestick Chart - Last Hour',
         ylabel='Price ($)',
         volume=True,
         ylabel_lower='Volume',
         figratio=(12, 6),
         mav=(3,6,9))  # Moving averages


### Understanding the Plotting Code

- **mpf.plot() Function**: This function from `mplfinance` is used to plot the candlestick chart. We pass our DataFrame `df` as the first argument.
  
- **type='candle'**: Specifies that we want to plot a candlestick chart.
  
- **style='charles'**: Chooses a predefined style for the chart. `mplfinance` offers several styles; 'charles' is just one example.
  
- **title, ylabel, and ylabel_lower**: These parameters set the title of the chart and labels for the y-axes. Since we're plotting volume as well, `ylabel_lower` is used for the volume subplot.
  
- **volume=True**: This includes a volume chart beneath the candlestick chart.
  
- **figratio**: Sets the width and height ratio of the figure. This can be adjusted based on your preference or to fit the data better.
  
- **mav**: Specifies the periods for which to calculate and plot moving averages. This example plots moving averages for 3, 6, and 9 periods. You can adjust these values or omit this parameter if you don't need moving averages.

### Try it Out

Experiment with different styles, moving average periods, or even zooming in on specific dates to analyze the stock's performance more closely. `mplfinance` offers various customization options to explore:


In [ ]:
# Zooming in on a specific date range
mpf.plot(df['2024-02-01':'2024-03-31'], type='candle', style='charles', ... )



Replace the `...` with the rest of the parameters from the previous example or try new ones to customize the plot further. This visual analysis can provide valuable insights into market trends and help inform your model development in the next stages.

## 3) Building the AI for Stock Price Prediction (Multi-variable LSTM Model)

Long Short-Term Memory (LSTM) networks are a type of recurrent neural network (RNN) capable of learning order dependence in sequence prediction problems. This makes them ideal for time series forecasting like stock price predictions. In this tutorial, we'll create a multi-variable LSTM model to predict stock prices in the next minute using TensorFlow and Keras.

Before we be let's make sure we install a few more machine learning packages:

- `tensorflow`: Enables building and training complex neural network models for deep learning applications.
- `scikit-learn`: Provides a wide range of simple and efficient tools for data mining and data analysis, including preprocessing and model evaluation.

In [ ]:
! pip install tensorflow scikit-learn


### Overview

Our dataset includes variables such as open, high, low, close prices, volume, transactions, and volume-weighted average price (VWAP). We'll use these features to predict the closing price in the next minute.



### Preparing the Data

First, we need to preprocess our data to fit the LSTM model. This involves scaling the data, creating a time series dataset with the appropriate sequence length, and splitting the dataset into training and test sets.



#### Scaling the Data

LSTMs are sensitive to the scale of the input data. It's common practice to normalize or standardize the data before training.


In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Select features to scale
features = ['open', 'high', 'low', 'close', 'volume', "transactions", "vwap"]
data = df[features]

# Use MinMaxScaler to scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Convert scaled data back to a DataFrame for easier manipulation
scaled_df = pd.DataFrame(scaled_data, columns=features)


**Summary**:
This code snippet is an essential step in data preprocessing for machine learning models, especially when dealing with time series data like stock prices. Scaling the data to a common scale allows the model to train more effectively, as it ensures that no single feature will dominate the learning process due to its scale.

<details>

<summary>💡 Click me for more details on the code</summary>

This code is designed to scale the features of a stock price dataset, preparing it for input into machine learning models, such as the LSTM model for predicting future stock prices. Let's break down each part of the code for a clearer understanding:

1. **Importing Required Libraries**:
   - `from sklearn.preprocessing import MinMaxScaler`: Imports the `MinMaxScaler` class from scikit-learn, a popular machine learning library. `MinMaxScaler` is used for scaling features to a given range.
   - `import numpy as np`: Imports the NumPy library, which is fundamental for scientific computing in Python. NumPy provides support for large, multi-dimensional arrays and matrices, along with a collection of mathematical functions to operate on these arrays.

2. **Selecting Features to Scale**:
   - `features = ['open', 'high', 'low', 'close', 'volume', "transactions", "vwap"]`: Defines a list of column names from the DataFrame `df` that will be scaled. These columns represent different attributes of stock prices, including the opening price (`open`), the highest price in the interval (`high`), the lowest price in the interval (`low`), the closing price (`close`), the number of shares traded (`volume`), the number of transactions (`transactions`), and the volume-weighted average price (`vwap`).

3. **Extracting Data for Scaling**:
   - `data = df[features]`: Extracts the columns specified in the `features` list from the DataFrame `df`. This creates a new DataFrame `data` containing only the columns that need to be scaled.

4. **Scaling the Data**:
   - `scaler = MinMaxScaler(feature_range=(0, 1))`: Creates an instance of the `MinMaxScaler` class. The parameter `feature_range=(0, 1)` specifies that the scaled values should fall within the range of 0 to 1.
   - `scaled_data = scaler.fit_transform(data)`: The `fit_transform` method computes the minimum and maximum values of `data` to perform the scaling (fit part) and then scales the `data` (transform part). The result is a NumPy array `scaled_data` where each original value in `data` is scaled to a value between 0 and 1.

5. **Converting Scaled Data Back to DataFrame**:
   - `scaled_df = pd.DataFrame(scaled_data, columns=features)`: Converts the scaled data, which is a NumPy array, back into a pandas DataFrame for easier manipulation in future steps. The `columns=features` argument ensures that the columns in `scaled_df` have the same names as the original DataFrame `df`, maintaining consistency and making it easier to understand which column represents what feature.

</details>


#### Creating Sequences

Next, we'll create sequences of data points to use as inputs for the LSTM. Each input sequence will be used to predict the closing price in the next minute.


In [ ]:
def create_sequences(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:(i + sequence_length)])
        y.append(data[i + sequence_length, 3])  # Assuming 'close' is at index 3
    return np.array(X), np.array(y)

sequence_length = 60  # Number of minutes to use for prediction
X, y = create_sequences(scaled_df.values, sequence_length)



**Summary**:
This code prepares the input and output data for training an LSTM network by organizing the scaled time series data into sequences. Each sequence represents a fixed period (in this case, 60 minutes) used by the network to predict the closing price in the next time step. This approach is crucial for capturing temporal dependencies in time series data, enabling more accurate predictions in tasks like stock price forecasting.

<details>

<summary>💡 Click me for more details on the code</summary>

This code snippet is focused on transforming the scaled stock price data into sequences that can be used for training a Long Short-Term Memory (LSTM) network. LSTM networks require input data in the form of sequences for making predictions based on time series data. Here's a detailed explanation of each part of the code:

1. **Defining the `create_sequences` Function**:
   - The function `create_sequences(data, sequence_length)` takes two parameters: `data`, which is a dataset containing scaled features, and `sequence_length`, which defines the length of the sequences to be created. The `sequence_length` is essentially the number of time steps the LSTM will look back to make a prediction.

2. **Initializing Empty Lists for Sequences and Labels**:
   - `X, y = [], []`: Initializes two empty lists, `X` and `y`. `X` will store the input sequences, and `y` will store the corresponding labels (or targets) for each sequence. In the context of stock price prediction, a sequence in `X` consists of stock prices and other features over a specified period, while the corresponding label in `y` is the price we want to predict at the next time step.

3. **Creating Sequences and Corresponding Labels**:
   - The `for` loop iterates through the `data`, creating sequences of length `sequence_length` and appending them to `X`. For each sequence in `X`, it also appends the next value of the closing price (assumed to be at index 3 in the `data` array) to `y` as the label.
   - `data[i:(i + sequence_length)]`: This slice of `data` takes `sequence_length` consecutive values starting from index `i`, creating a sequence that is added to `X`.
   - `data[i + sequence_length, 3]`: This accesses the closing price (`close`), which is immediately after the end of the current sequence, and adds it to `y` as the label for the sequence. The assumption here is that the 'close' price is at index 3 of the `data` array.

4. **Converting Lists to NumPy Arrays**:
   - `return np.array(X), np.array(y)`: The function returns two NumPy arrays, `X` and `y`. Converting the lists to NumPy arrays is important for compatibility with TensorFlow and Keras, which are used to build and train the LSTM model. NumPy arrays provide efficient storage and computation capabilities, especially for large datasets.

5. **Specifying the Sequence Length**:
   - `sequence_length = 60`: This sets the `sequence_length` to 60, meaning each sequence will contain data for 60 minutes. This parameter can be adjusted based on the frequency of the data and the specific requirements of the prediction task.

6. **Generating Sequences and Labels**:
   - `X, y = create_sequences(scaled_df.values, sequence_length)`: Calls the `create_sequences` function with the scaled data (`scaled_df.values`) and the specified `sequence_length` to generate the input sequences (`X`) and their corresponding labels (`y`).

</details>

#### Splitting the Dataset

Split the dataset into training and test sets. Typically, we use the most recent data for testing.


In [ ]:
train_split = int(0.8 * len(X))
X_train, X_test = X[:train_split], X[train_split:]
y_train, y_test = y[:train_split], y[train_split:]

**Summary**:
The purpose of this code is to partition the dataset into training and test sets, a common practice in machine learning to evaluate model performance. By training the model on a subset of the data (`X_train` and `y_train`) and testing it on unseen data (`X_test` and `y_test`), we can assess how well the model generalizes to new, unseen data points. This step is essential for avoiding overfitting, where a model might perform well on training data but poorly on new data, and for ensuring that the model can make accurate predictions in real-world situations.

<details>

<summary>💡 Click me for more details on the code</summary>

This code snippet is about splitting the sequences of stock price data into training and test sets. This is a crucial step in preparing the data for training and evaluating a machine learning model, such as an LSTM network for time series prediction. Here's a detailed breakdown of each part of the code:

1. **Determining the Training Set Size**:
   - `train_split = int(0.8 * len(X))`: This line calculates the size of the training set. It multiplies the total number of sequences (`len(X)`) by 0.8 to use 80% of the data for training. The `int()` function is used to ensure that the result is an integer, as the number of sequences must be a whole number. This split ratio can be adjusted based on the specific requirements of the model or to ensure that the model has enough data for effective training and validation.

2. **Splitting the Sequences into Training and Test Sets**:
   - `X_train, X_test = X[:train_split], X[train_split:]`: This line splits the input sequences (`X`) into a training set (`X_train`) and a test set (`X_test`). The training set contains the first 80% of the sequences, as determined by `train_split`, while the test set contains the remaining 20%. This split allows the model to learn from the training data and then be evaluated on unseen data from the test set to gauge its predictive performance.
   
   - `y_train, y_test = y[:train_split], y[train_split:]`: Similarly, this line splits the labels (`y`) into a training set (`y_train`) and a test set (`y_test`) using the same `train_split` index. This ensures that each input sequence in `X_train` and `X_test` has a corresponding label in `y_train` and `y_test`, respectively.

</details>


### Building the LSTM Model

Now, let's construct our LSTM model using TensorFlow and Keras.



In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam


model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    LSTM(50, return_sequences=True),
    Dropout(0.2),
    LSTM(50),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')


**Summary**:
This model is structured to capture the temporal dependencies in the time series data of stock prices through its LSTM layers, while also employing regularization via Dropout layers to mitigate the risk of overfitting. The final Dense layer outputs the predicted stock price, and the model is compiled with a configuration suited for regression tasks. Training this model involves fitting it to a sequence of stock prices and volumes, aiming to predict the stock price at the next minute accurately.

<details>

<summary>💡 Click me for more details on the code</summary>

This code snippet is focused on building a Sequential model using TensorFlow's Keras API, specifically designed for a time series prediction task like forecasting stock prices. The model employs Long Short-Term Memory (LSTM) layers, which are well-suited for learning from sequences of data, along with Dropout layers to prevent overfitting. Here's a detailed explanation of each part:

1. **Importing Required Modules**:
   - `from tensorflow.keras.models import Sequential`: Imports the `Sequential` model class from TensorFlow's Keras API. A `Sequential` model is a linear stack of layers.
   - `from tensorflow.keras.layers import LSTM, Dense, Dropout`: Imports the `LSTM`, `Dense`, and `Dropout` layer classes. `LSTM` layers are used for sequence prediction problems, `Dense` layers are fully connected neural network layers, and `Dropout` layers help prevent overfitting by randomly setting input units to 0 during training.
   - `from tensorflow.keras.optimizers import Adam`: Imports the `Adam` optimizer, a popular algorithm for training neural networks.

2. **Building the Sequential Model**:
   - `model = Sequential([...])`: Initializes a new Sequential model. The model is defined by passing a list of layers to the `Sequential` constructor. This list specifies the architecture of the neural network.

3. **Defining the Model Architecture**:
   - The model architecture consists of three LSTM layers and three Dropout layers, ending with a Dense layer.
   - `LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]))`: The first layer is an LSTM layer with 50 units. The `return_sequences=True` argument is necessary for stacking LSTM layers so that the subsequent LSTM layer receives sequences as input. The `input_shape` is specified according to the shape of the training data, allowing the model to know the input dimensionality.
   - `Dropout(0.2)`: Following each LSTM layer (except the last), a Dropout layer is added with a rate of 0.2, meaning 20% of the input units are randomly set to 0 during training, which helps in preventing overfitting.
   - `Dense(1)`: The final layer is a Dense layer with a single unit. In the context of stock price prediction, this output corresponds to the predicted value of the stock price at the next time step.

4. **Compiling the Model**:
   - `model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')`: The model is compiled with the Adam optimizer, with a learning rate of 0.001, and the mean squared error loss function. The choice of loss function is appropriate for regression problems, where the goal is to minimize the difference between the predicted and actual values.

</details>

### Model Training

Train the model using the training data. Depending on your dataset size, this might take some time.


In [ ]:
import os

def create_unique_file(base_filename, extension=''):
    counter = 1
    filename = f"{base_filename}{extension}"
    while os.path.exists(filename):
        filename = f"{base_filename}_{counter}{extension}"
        counter += 1

    return filename

history = model.fit(X_train, y_train, epochs=7, batch_size=32, validation_split=0.1, verbose=1)

# Create a unique file name for the model
filename = create_unique_file(f"{symbol}_lstm_model", ".keras")

# Save your model for later
model.save(filename)


**Summary**: This code snippet efficiently trains a neural network model and then saves it in a way that ensures the filename is unique. This is particularly useful when training multiple models in a script or when working in directories with many files, as it avoids accidental data loss due to file overwriting.

<details>

<summary>💡 Click me for more details on the code</summary>

This code snippet demonstrates how to train a machine learning model using TensorFlow and Keras, and then save the trained model to a unique file for later use. It also includes a custom function to ensure that the saved file does not overwrite any existing files. Let's break down the code for better understanding:

**Part 1: Training the Model**

- `history = model.fit(X_train, y_train, epochs=7, batch_size=32, validation_split=0.1, verbose=1)`: This line trains the model on the training data (`X_train`, `y_train`) over 7 epochs, with a batch size of 32. During training, 10% of the training data is held back as a validation set (`validation_split=0.1`) to monitor the model's performance on unseen data. The `verbose=1` argument specifies that the training process will output detailed information about its progress after each epoch.

**Part 2: Defining a Function to Create a Unique Filename**

- `import os`: Imports the `os` module, which provides functions for interacting with the operating system, including checking whether a file exists.

- `def create_unique_file(base_filename, extension='')`: Defines a function that takes a base filename and an optional extension as arguments. The purpose of this function is to generate a unique filename by appending a counter to the base filename if a file with the proposed name already exists.

  - `counter = 1`: Initializes a counter used to generate unique filenames if needed.
  - `filename = f"{base_filename}{extension}"`: Constructs the initial filename using the provided base filename and extension.
  - `while os.path.exists(filename)`: Checks if a file with the current filename exists. If it does, the loop constructs a new filename with an appended counter (`filename = f"{base_filename}_{counter}{extension}"`) and increments the counter. This loop continues until a unique filename is found.
  - `return filename`: Returns the unique filename.

**Part 3: Saving the Trained Model**

- `filename = create_unique_file(f"{symbol}_lstm_model", ".keras")`: Calls the `create_unique_file` function with the base filename as `"{symbol}_lstm_model"` and the extension as `".keras"`. The `symbol` variable is assumed to be a string that represents the stock symbol the model was trained on. This step ensures that the model is saved to a unique file, preventing any existing files from being overwritten.

- `model.save(filename)`: Saves the trained model to the file with the generated unique filename. This allows the model to be loaded and reused later, without needing to retrain.

</details>


### Predicting and Evaluating the Model

Finally, use the model to make predictions on the test set and evaluate its performance.


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

# Evaluate the model on the test data
test_loss = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {test_loss}")

# Make predictions
predictions = model.predict(X_test)

# Calculate performance metrics
mse = mean_squared_error(y_test, predictions)
rmse = sqrt(mse)
mae = mean_absolute_error(y_test, predictions)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")



**Summary**: The code snippet evaluates a machine learning model's performance on unseen test data by calculating the test loss, making predictions, and quantifying prediction errors using key metrics (MSE, RMSE, MAE). These steps are essential for assessing the model's predictive accuracy and for guiding improvements.

<details>

<summary>💡 Click me for more details on the code</summary>

This code snippet is focused on evaluating the performance of a trained neural network model, specifically for a regression task such as stock price prediction. It evaluates the model's loss on the test dataset, makes predictions, and calculates key performance metrics. Here's a breakdown of each part:

**Part 1: Importing Required Libraries**

- `import matplotlib.pyplot as plt`: Imports the Matplotlib library for plotting, which is not directly used in the given snippet but is commonly used for visualizing data and model performance.
- `from sklearn.metrics import mean_squared_error, mean_absolute_error`: Imports functions to calculate the mean squared error (MSE) and mean absolute error (MAE) from scikit-learn, a machine learning library. These metrics are used to quantify the model's prediction errors.
- `from math import sqrt`: Imports the `sqrt` function from the math module to calculate the square root, which is used for computing the root mean squared error (RMSE).

**Part 2: Evaluating the Model**

- `test_loss = model.evaluate(X_test, y_test, verbose=0)`: This line evaluates the trained model on the test dataset (`X_test`, `y_test`) without printing the evaluation output (`verbose=0`). The evaluation returns the loss value (in this case, mean squared error, as specified during model compilation) on the test data. The loss value quantifies how well the model's predictions match the actual labels.

- `print(f"Test Loss: {test_loss}")`: Prints the loss of the model on the test data, providing a high-level quantification of the model's prediction error.

**Part 3: Making Predictions**

- `predictions = model.predict(X_test)`: Uses the trained model to make predictions on the test dataset. The output is a numpy array of predicted values corresponding to the input samples in `X_test`.

**Part 4: Calculating Performance Metrics**

- `mse = mean_squared_error(y_test, predictions)`: Calculates the mean squared error between the actual labels (`y_test`) and the predicted values (`predictions`). MSE is the average of the squares of the errors and is commonly used to measure the accuracy of continuous variables.

- `rmse = sqrt(mse)`: Calculates the root mean squared error, which is the square root of MSE. RMSE is a measure of the accuracy that is in the same units as the response variable. It's particularly useful because it gives a relatively high weight to large errors.

- `mae = mean_absolute_error(y_test, predictions)`: Calculates the mean absolute error, which is the average of the absolute differences between the predictions and actual values. Unlike MSE or RMSE, MAE provides a linear measure of prediction accuracy.

- `print` statements: These lines print out the calculated MSE, RMSE, and MAE, providing a detailed overview of the model's prediction performance. These metrics are crucial for understanding the model's accuracy and for comparing it with other models.

</details>

### Visualizing Predictions

It's helpful to visualize the predictions against the actual closing prices.


In [ ]:
import matplotlib.pyplot as plt

# Plot predictions vs actual values
plt.figure(figsize=(10, 6))
plt.plot(y_test, label='Actual')
plt.plot(predictions, label='Predicted')
plt.title('Model Predictions vs Actual Values')
plt.xlabel('Time')
plt.ylabel('Close Price')
plt.legend()
plt.show()


**Summary**: This code snippet effectively visualizes the comparison between the actual closing prices and the predicted prices generated by a machine learning model. Visual comparison helps in quickly assessing the model's predictive performance and understanding how closely the predictions align with the actual values over the given time period.

<details>

<summary>💡 Click me for more details on the code</summary>

This code snippet is about visualizing the comparison between actual values and model predictions, specifically targeting a regression problem like stock price forecasting. By plotting both sets of data on the same graph, it provides an intuitive understanding of the model's performance. Here's a step-by-step explanation:

### Part 1: Importing Matplotlib

- `import matplotlib.pyplot as plt`: This imports Matplotlib's `pyplot` module, a collection of functions that make Matplotlib work like MATLAB, allowing for interactive plots and simple visualizations. Matplotlib is a widely used Python library for data visualization.

### Part 2: Setting Up the Plot

- `plt.figure(figsize=(10, 6))`: Creates a new figure for plotting with a specified size of 10 inches in width and 6 inches in height. The `figsize` parameter ensures that the plot has enough space to clearly display the data without squeezing the labels and titles.

### Part 3: Plotting Data

- `plt.plot(y_test, label='Actual')`: Plots the actual values (`y_test`) on the figure. These values represent the true closing prices from the test dataset. The `label='Actual'` argument assigns a label to this line for identification in the legend.

- `plt.plot(predictions, label='Predicted')`: Plots the predicted values obtained from the model on the same figure. The predictions are the model's output based on the input features from the test dataset. Similar to the actual values, a label is assigned for identification in the legend.

### Part 4: Customizing the Plot

- `plt.title('Model Predictions vs Actual Values')`: Sets the title of the plot to "Model Predictions vs Actual Values," which describes the purpose of the plot — comparing the model's predictions against the actual closing prices.

- `plt.xlabel('Time')`: Labels the x-axis as "Time." In this context, "Time" represents the sequence of data points, although it does not specify the exact time frame due to the generic nature of the plot. For more detailed time series plots, you might consider plotting actual datetime values on the x-axis.

- `plt.ylabel('Close Price')`: Labels the y-axis as "Close Price," indicating that the values being plotted represent the closing prices of the stock.

- `plt.legend()`: Adds a legend to the plot. The legend uses the labels defined earlier (`'Actual'` and `'Predicted'`) to distinguish between the two lines plotted on the graph.

### Part 5: Displaying the Plot

- `plt.show()`: Displays the figure. This command renders the plot and shows it in a window. In Jupyter notebooks and other similar environments, this will display the plot inline.

</details>



### Conclusion

You've now built and trained a multi-variable LSTM model to predict stock prices. Remember, stock market prediction is inherently uncertain and influenced by many external factors. Always consider this when evaluating model performance.

### What's next? 🧪 Experiment!

This is just the starting point. Try changing the parameters we used to build and train the model. Here are some suggestions:

**In the build phase:**
- **Layers**: Experiment with adding or removing layers. A comprehensive list of available layers can be found on the Keras documentation website at [https://keras.io/api/layers/](https://keras.io/api/layers/). Each layer type offers different functionalities, such as `Conv1D` for convolutional operations on sequences or `Bidirectional` to make your LSTM layers process the sequences in both directions.
- **Layer Parameters**: Try modifying the parameters of the existing layers. For example, you can adjust the number of units in LSTM layers or the dropout rate in Dropout layers to see how they impact model performance.

**In the train phase:**
- **Epochs**: Increase this to add more training phases. More epochs mean the model will have more opportunities to learn from the data, but watch out for overfitting.
- **Batch Size**: Experiment with the batch size. A smaller batch size might increase training time but can lead to a more stable and fine-tuned learning process.
- **Validation Split**: Adjust the percentage of data used for validation during training. Finding the right balance between training and validation data can help in building a model that generalizes well.

**Evaluation and Metrics:**
- Consider using additional metrics for evaluation, such as `R^2` score, to get a better sense of how well your model is performing.
- Plot learning curves to visualize the model's learning process over time and identify if and when the model starts overfitting.

By experimenting with different configurations, you'll gain a deeper
understanding of how various aspects of the model and the training process
affect performance. Happy modeling!